In [ ]:
!pip install optuna

In [ ]:
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, KFold

import optuna
from optuna.integration import CatBoostPruningCallback

from catboost import CatBoostClassifier

In [ ]:
df = pd.read_csv('sample_data/final_df_sleep_v0.csv')
df

In [ ]:
plt.figure(figsize = (20, 20))
sns.heatmap(df.corr(), annot = True, cmap = 'viridis')

In [ ]:
df.isna().sum()

In [ ]:
df = df.drop(columns = ['time'])
df

In [ ]:
X = df.drop(columns = ['target'])
y = df['target']

#Catboost + optuna

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 0.3, 
                                                    random_state = 42, 
                                                    shuffle = True)

In [ ]:
def objective_catboost(trial, X, y):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size = 0.25)

    param_grid_catboost = { 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.01, 0.1, log = True),
                            'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
                            'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
                            'iterations': trial.suggest_int('iterations', 100, 1200),
                            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 10, log = True),
                            'border_count': trial.suggest_int('border_count', 32, 255),
                            'random_strength': trial.suggest_float("random_strength", 1e-8, 10.0, log = True),
                            'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
                            'od_wait': trial.suggest_int('od_wait', 10, 50),
                            'depth': trial.suggest_int('depth', 1, 12),
                            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
                            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 15),
                            'eval_metric': 'BalancedAccuracy',
                            'task_type': 'CPU',
                            'auto_class_weights' : 'Balanced'}

    if param_grid_catboost['bootstrap_type'] == 'Bayesian':
        param_grid_catboost['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif param_grid_catboost['bootstrap_type'] == 'Bernoulli':
        param_grid_catboost['subsample'] = trial.suggest_float('subsample', 0.1, 1, log = True)
    
    pruning_callback = CatBoostPruningCallback(trial, 'BalancedAccuracy')

    cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(train_x, train_y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        catboost_model = CatBoostClassifier(**param_grid_catboost)
        
        catboost_model.fit(X_train, y_train, eval_set=[(valid_x, valid_y)], early_stopping_rounds = 100, callbacks = [pruning_callback])
    
    pruning_callback.check_pruned()

    return balanced_accuracy_score(catboost_model.predict(X_test), y_test).mean()

In [ ]:
study_catboost = optuna.create_study(pruner = optuna.pruners.MedianPruner(n_warmup_steps = 5), direction = 'maximize')
func_catboost = lambda trial: objective_catboost(trial, X_train, y_train)
study_catboost.optimize(func_catboost, n_trials = 3)

In [ ]:
optuna.visualization.plot_optimization_history(study_catboost)

In [ ]:
optuna.visualization.plot_param_importances(study_catboost)

In [ ]:
study_catboost.best_params

In [ ]:
cv = KFold(n_splits = 5, shuffle = True)
 
for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    catboost = CatBoostClassifier(**study_catboost.best_params)

    catboost_model = catboost.fit(X_train, y_train)   
    
print(balanced_accuracy_score(catboost_model.predict(X_test), y_test))

In [ ]:
catboost_preds_test = catboost_model.predict(X_test)

catboost_AS_test = accuracy_score(catboost_preds_test, y_test)
catboost_BAS_test = balanced_accuracy_score(catboost_preds_test, y_test)
catboost_f1_weighted_test = f1_score(catboost_preds_test, y_test, average = 'weighted')
catboost_roc_auc_score_test = roc_auc_score(catboost_preds_test, y_test)
catboost_mcc_test = matthews_corrcoef(catboost_preds_test, y_test)

catboost_test_scores_dict = {'Accuracy' : catboost_AS_test,
                             'Balanced accuracy' : catboost_BAS_test,
                             'F1 score' : catboost_f1_weighted_test,
                             'ROC auc score' : catboost_roc_auc_score_test,
                             'Matthews correlation coefficient' : catboost_mcc_test}

catboost_test_scores_df = pd.DataFrame.from_dict(catboost_test_scores_dict, orient = 'index', columns = ['Scores'])

catboost_test_scores_df

In [ ]:
cm = confusion_matrix(catboost_model.predict(X_test), y_test)
cm

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['Non-REM sleep', 'REM sleep'])
disp.plot()
plt.show()

In [ ]:
catboost_model.save_model('catboost_sleep.json')